# YOLO v8 Object Detection


In [2]:
import os
import time
import cv2
import numpy as np
import math
import matplotlib.pyplot as plt
from model.yolo_model import YOLO

In [15]:
def process_image(img):
    """Resize, reduce and expand image.

    # Argument:
        img: original image.

    # Returns
        image: ndarray(64, 64, 3), processed image.
    """
    image = cv2.resize(img, (1920, 1080),
                       interpolation=cv2.INTER_CUBIC)
    image = np.array(image, dtype='float32')
    #image /= 255.
    #image = np.expand_dims(image, axis=0)

    return image

In [4]:
def get_classes(file):
    """Get classes name.

    # Argument:
        file: classes name for database.

    # Returns
        class_names: List, classes name.

    """
    with open(file) as f:
        class_names = f.readlines()
    class_names = [c.strip() for c in class_names]

    return class_names

In [5]:
def draw(image, boxes, all_classes):
    """Draw the boxes on the image.

    # Argument:
        image: original image.
        boxes: ndarray, boxes of objects.
        classes: ndarray, classes of objects.
        scores: ndarray, scores of objects.
        all_classes: all classes name.
        
    """
    
    for box in boxes:
        x, y, w, h = box.xyxy[0]
        print(x,y,w,h)
        x, y, w, h = int(x), int(y), int(w), int(h)
        
        cv2.rectangle(image, (x, y), (w, h), (255, 0, 0), 2)
        
        scores = math.ceil((box.conf[0]*100))/100
        cls = int(box.cls[0])
        
        cv2.putText(image, '{0} {1:.2f}'.format(all_classes[cls], scores),
                    (x, y),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    0.6, (0, 0, 255), 1,
                    cv2.LINE_AA)

        print('class: {0}, score: {1:.2f}'.format(all_classes[cls], scores))
        print('box coordinate x,y,w,h: {0}'.format(box))

    print()

In [6]:
def detect_image(image, yolo, all_classes):
    """Use yolo v3 to detect images.

    # Argument:
        image: original image.
        yolo: YOLO, yolo model.
        all_classes: all classes name.

    # Returns:
        image: processed image.
    """
    pimage = process_image(image)

    start = time.time()
    results = yolo.predict(pimage, image.shape)
    end = time.time()

    print('time: {0:.2f}s'.format(end - start))
    

    for result in results:
        boxes = result.boxes
        
        filtered_boxes = [box for box in boxes if math.ceil((box.conf[0]*100))/100 >= 0.5]
        
        if filtered_boxes:
            draw(image, filtered_boxes, all_classes)

    return image

In [7]:
def detect_video(video, yolo, all_classes):
    """Use yolo v3 to detect video.

    # Argument:
        video: video file.
        yolo: YOLO, yolo model.
        all_classes: all classes name.
    """
    video_path = os.path.join("videos", "test", video)
    camera = cv2.VideoCapture(video_path)
    cv2.namedWindow("detection", cv2.WINDOW_AUTOSIZE)

    # Prepare for saving the detected video
    sz = (int(camera.get(cv2.CAP_PROP_FRAME_WIDTH)),
        int(camera.get(cv2.CAP_PROP_FRAME_HEIGHT)))
    fourcc = cv2.VideoWriter_fourcc(*'mpeg')

    
    vout = cv2.VideoWriter()
    vout.open(os.path.join("videos", "res", video), fourcc, 20, sz, True)

    while True:
        res, frame = camera.read()

        if not res:
            break

        image = detect_image(frame, yolo, all_classes)
        cv2.imshow("detection", image)

        # Save the video frame by frame
        vout.write(image)

        if cv2.waitKey(110) & 0xff == 27:
                break

    vout.release()
    camera.release()
    cv2.destroyAllWindows()
    

In [8]:
#pip install ultralytics

In [9]:
#localde modeli bu şekilde eğittim
"""
from ultralytics import YOLO
import torch

def main():

    if torch.cuda.is_available():
        print(f"GPU is available. Using {torch.cuda.get_device_name(0)}")
    else:
        print("GPU is not available. Using CPU")

    model = YOLO('yolov8n.pt')

    data = 'VOC.yaml'

    model.train(data=data, epochs=50, batch=8, imgsz=640)

if __name__ == '__main__':
    main()

model = YOLO('runs/detect/train8/weights/best.pt')
model.save('yolo_model.h5')
"""

'\nfrom ultralytics import YOLO\nimport torch\n\ndef main():\n\n    if torch.cuda.is_available():\n        print(f"GPU is available. Using {torch.cuda.get_device_name(0)}")\n    else:\n        print("GPU is not available. Using CPU")\n\n    model = YOLO(\'yolov8n.pt\')\n\n    data = \'VOC.yaml\'\n\n    model.train(data=data, epochs=50, batch=8, imgsz=640)\n\nif __name__ == \'__main__\':\n    main()\n\nmodel = YOLO(\'runs/detect/train8/weights/best.pt\')\nmodel.save(\'yolo_model.h5\')\n'

In [10]:
from ultralytics import YOLO
from tensorflow.keras.models import load_model

model = YOLO('runs/detect/train8/weights/best.pt')

file = 'data/voc_classes.txt'
all_classes = get_classes(file)

### Detecting Images

In [14]:
f = 'person.jpg'
path = 'images/test/'+f
image = cv2.imread(path)
image = detect_image(image, model, all_classes)
cv2.imwrite('images/res/' + f, image)

time: 0.00s

0: 448x640 1 dog, 1 horse, 1 person, 316.7ms
tensor(62.7650) tensor(262.1987) tensor(202.5750) tensor(346.0482)
class: dog, score: 0.94
box coordinate x,y,w,h: ultralytics.engine.results.Boxes object with attributes:

cls: tensor([11.])
conf: tensor([0.9345])
data: tensor([[ 62.7650, 262.1987, 202.5750, 346.0482,   0.9345,  11.0000]])
id: None
is_track: False
orig_shape: (424, 640)
shape: torch.Size([1, 6])
xywh: tensor([[132.6700, 304.1235, 139.8100,  83.8495]])
xywhn: tensor([[0.2073, 0.7173, 0.2185, 0.1978]])
xyxy: tensor([[ 62.7650, 262.1987, 202.5750, 346.0482]])
xyxyn: tensor([[0.0981, 0.6184, 0.3165, 0.8162]])
tensor(189.9857) tensor(99.1192) tensor(274.1155) tensor(373.3610)
class: person, score: 0.92
box coordinate x,y,w,h: ultralytics.engine.results.Boxes object with attributes:

cls: tensor([14.])
conf: tensor([0.9114])
data: tensor([[189.9857,  99.1192, 274.1155, 373.3610,   0.9114,  14.0000]])
id: None
is_track: False
orig_shape: (424, 640)
shape: torch.Size([

True

In [55]:
#Karşılaştığım hataları manuel olarak burda test ederek buldum.
"""

image=cv2.imread('images/test/person.jpg')
image = cv2.resize(image, (640, 640),
                       interpolation=cv2.INTER_CUBIC)
image = np.array(image, dtype='float32')
#image /= 255.
#image = np.expand_dims(image, axis=0)
results = model.predict(image, image.shape)

for result in results:
    boxes = result.boxes
        
    if boxes:
        draw(image, boxes, all_classes)
    

"""


"\n\nimage=cv2.imread('images/test/person.jpg')\nimage = cv2.resize(image, (640, 640),\n                       interpolation=cv2.INTER_CUBIC)\nimage = np.array(image, dtype='float32')\n#image /= 255.\n#image = np.expand_dims(image, axis=0)\nresults = model.predict(image, image.shape)\n\nfor result in results:\n    boxes = result.boxes\n        \n    if boxes:\n        draw(image, boxes, all_classes)\n    \n\n"

# Detecting on Video

In [17]:
# # detect videos one at a time in videos/test folder    
video = 'sun1.mp4'
detect_video(video, model, all_classes)

time: 0.00s

0: 384x640 2 cars, 133.5ms
tensor(1062.2578) tensor(769.3348) tensor(1143.1732) tensor(836.6492)
class: car, score: 0.85
box coordinate x,y,w,h: ultralytics.engine.results.Boxes object with attributes:

cls: tensor([6.])
conf: tensor([0.8494])
data: tensor([[1.0623e+03, 7.6933e+02, 1.1432e+03, 8.3665e+02, 8.4943e-01, 6.0000e+00]])
id: None
is_track: False
orig_shape: (1080, 1920)
shape: torch.Size([1, 6])
xywh: tensor([[1102.7156,  802.9920,   80.9154,   67.3143]])
xywhn: tensor([[0.5743, 0.7435, 0.0421, 0.0623]])
xyxy: tensor([[1062.2578,  769.3348, 1143.1732,  836.6492]])
xyxyn: tensor([[0.5533, 0.7123, 0.5954, 0.7747]])

Speed: 0.0ms preprocess, 133.5ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)
time: 0.00s

0: 384x640 1 car, 92.2ms
tensor(1062.6802) tensor(768.9102) tensor(1143.4700) tensor(835.5809)
class: car, score: 0.84
box coordinate x,y,w,h: ultralytics.engine.results.Boxes object with attributes:

cls: tensor([6.])
conf: tensor([0.8344])
da